# Coupled Quadratic Program

## Introduction

A quadratic program (QP) is an optimization problem with a quadratic objective and affine equality and inequality constraints. We consider a QP in which $L$ variable blocks are coupled through a set of $s$ linear constraints, represented as

$$\begin{array}{ll}
\text{minimize} & \sum_{l=1}^L z_l^TQ_lz_l+c_l^Tz_l\\
\text{subject to} & F_lz_l\leq d_l,\quad l=1,\dots,L,\\
& \sum_{l=1}^LG_lz_l=h
\end{array}$$

with respect to $z=(z_1,\dots,z_L)$, where $z_l \in \mathbf{R}^{q_l}, Q_l\in \mathbf{S}_{+}^{q_l}$ (the set of positive semidefinite matrices), $c_l\in\mathbf{R}^{q_l}, F_l \in \mathbf{R}^{p_l \times q_l}, d_l \in \mathbf{R}^{p_l}, G_l \in \mathbf{R}^{s \times q_l}$, and $h \in \mathbf{R}^s$ for $l = 1,\ldots,L$.

## Reformulate Problem

Given a set $C \subseteq \mathbf{R}^q$, define the set indicator function $I_C: \mathbf{R}^q \rightarrow \mathbf{R} \cup \{\infty\}$ to be

$$I_C(x) = \begin{cases} 0 & x \in C \\ \infty & \text{otherwise}. \end{cases}$$

The coupled QP can be written in standard form with

$$f_i(x_i)=x_i^TQ_ix_i+c_i^Tx_i+I_{\{x\,:\,F_ix\leq d_i\}}(x_i), 
\quad i = 1,\ldots,L,$$
$$A = [G_1~\ldots~G_L], \quad b = h.$$

## Generate Data

We solve an instance of this problem with $L = 4, s= 10, q_l = 30$, and $p_l = 50$ for $l = 1,\ldots,L$. The entries of $c_l \in \mathbf{R}^{q_l}$, $F_l \in \mathbf{R}^{p_l \times q_l}$, $G_l \in \mathbf{R}^{s \times q_l}$, $\tilde z_l \in \mathbf{R}^{q_l}$, and $H_l \in \mathbf{R}^{q_l \times q_l}$ are all drawn IID from $N(0,1)$. We then form $d_l = F_l\tilde z_l + 0.1, Q_l = H_l^TH_l$, and $h = \sum_{l=1}^L G_l\tilde z_l$.

In [1]:
import numpy as np

np.random.seed(1)

L = 4    # Number of blocks.
s = 10   # Number of coupling constraints.
ql = 30  # Variable dimension of each QP subproblem.
pl = 50  # Constraint dimension of each QP subproblem.

c_list = [np.random.randn(ql) for l in range(L)]
F_list = [np.random.randn(pl,ql) for l in range(L)]
G_list = [np.random.randn(s,ql) for l in range(L)]
z_tld_list = [np.random.randn(ql) for l in range(L)]
H_list = [np.random.randn(ql,ql) for l in range(L)]

d_list = [F_list[l].dot(z_tld_list[l]) + 0.1 for l in range(L)]
Q_list = [H_list[l].T.dot(H_list[l]) for l in range(L)]

G = np.hstack(G_list)
z_tld = np.hstack(z_tld_list)
h = G.dot(z_tld)

## Define Proximal Operator

A2DR requires us to provide a proximal oracle for each $f_i$, which computes $\mathbf{prox}_{tf_i}(v_i)$ given any $v_i \in \mathbf{R}^{q_i}$. To do this, we must solve the quadratic program

$$\begin{array}{ll}
\text{minimize} & x_i^T\left(Q_i+\frac{1}{2t}I\right)
x_i +(c_i-\frac{1}{t}v_i)^Tx_i\\
\text{subject to} & F_ix_i\leq d_i
\end{array}$$

with respect to $x_i \in \mathbf{R}^{q_i}$. 

There are many QP solvers available. In this example, we will use [OSQP](https://osqp.org/) called via the [CVXPY](https://www.cvxpy.org/) interface.

In [2]:
import cvxpy
from cvxpy import *

def prox_qp(v, t, Q, c, F, d):
    q = Q.shape[0]
    I = np.eye(q)
    
    # Construct problem.
    x = Variable(q)
    obj = quad_form(x, Q + I/(2*t)) + (c - v/t)*x
    constr = [F*x <= d]
    prob = Problem(Minimize(obj), constr)
    
    # Solve with OSQP.
    prob.solve(solver = "OSQP")
    return x.value

## Solve Problem

In [3]:
from a2dr import a2dr

# Convert problem to standard form.
def prox_qp_wrapper(l, Q_list, c_list, F_list, d_list):
    return lambda v, t: prox_qp(v, t, Q_list[l], c_list[l], F_list[l], d_list[l])

# Use "map" method to define list of proximal operators. This addresses the late binding issue:
# https://stackoverflow.com/questions/3431676/creating-functions-in-a-loop
# https://docs.python-guide.org/writing/gotchas/#late-binding-closures
prox_list = list(map(lambda l: prox_qp_wrapper(l, Q_list, c_list, F_list, d_list), range(L)))

# Solve with A2DR.
a2dr_result = a2dr(prox_list, G_list, h)
a2dr_x = a2dr_result["x_vals"]

# Compute objective and constraint violation.
a2dr_obj = np.sum([a2dr_x[l].dot(Q_list[l]).dot(a2dr_x[l]) + c_list[l].dot(a2dr_x[l]) for l in range(L)])
a2dr_constr_vio = [np.linalg.norm(np.maximum(F_list[l].dot(a2dr_x[l]) - d_list[l], 0))**2 for l in range(L)]
a2dr_constr_vio += [np.linalg.norm(G.dot(np.hstack(a2dr_x)) - h)**2]
a2dr_constr_vio_val = np.sqrt(np.sum(a2dr_constr_vio))

# Print solution.
print("Objective value:", a2dr_obj)
print("Constraint violation:", a2dr_constr_vio_val)

----------------------------------------------------------------------
a2dr v0.2.3.post1 - Prox-Affine Distributed Convex Optimization Solver
                       (c) Anqi Fu, Junzi Zhang
                      Stanford University   2019
----------------------------------------------------------------------
### Preconditioning starts ... ###
### Preconditioning finished.  ###
max_iter = 1000, t_init (after preconditioning) = 1.73
eps_abs = 1.00e-06, eps_rel = 1.00e-08, precond = True
ada_reg = True, anderson = True, m_accel = 10
lam_accel = 1.00e-08, aa_method = lstsq, D_safe = 1.00e+06
eps_safe = 1.00e-06, M_safe = 10
variables n = 120, constraints m = 10
nnz(A) = 1200
Setup time: 1.95e-02
----------------------------------------------------
 iter | total res | primal res | dual res | time (s)
----------------------------------------------------
     0| 2.81e+01    7.02e+00     2.72e+01   1.16e-01
   100| 1.64e-01    8.35e-02     1.41e-01   2.68e+00
   194| 1.13e-06    1.11e-06     1